In [ ]:
!pip install gradio

In [ ]:
!pip install automata-lib gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
!pip install gradio automata-lib

from automata.fa.dfa import DFA
import gradio as gr
# 1's Complement function
def ones_complement_dfa(binary_str):
    # Validate input
    if not all(bit in '01' for bit in binary_str):
        return "Invalid input! Enter a binary string (only 0s and 1s)."

    states = {'q0', 'q1'}
    alphabet = {'0', '1'}
    transitions = {
        'q0': {'0': 'q1', '1': 'q1'},
        'q1': {'0': 'q0', '1': 'q0'}
    }
    initial_state = 'q0'
    final_states = {'q0', 'q1'}

    dfa = DFA(
        states=states,
        input_symbols=alphabet,
        transitions=transitions,
        initial_state=initial_state,
        final_states=final_states
    )

    current_state = initial_state
    result = ""
    steps = ["Start at initial state: q0"]

    for bit in binary_str:
        new_bit = '1' if bit == '0' else '0'  # Flip the bit
        result += new_bit
        next_state = dfa.transitions[current_state][bit]
        steps.append(f"Input: '{bit}' -> Transition: {current_state} -> {next_state} (Bit flipped to '{new_bit}')")
        current_state = next_state  # Move to the next state

    steps.append(f"End in final state: {current_state} (1's Complement result: {result})")
    return result, "\n".join(steps)

# 2's Complement function
def twos_complement_dfa(binary_str):
    ones_comp_result, ones_steps = ones_complement_dfa(binary_str)
    steps = ones_steps.split("\n")
    steps.append("Transition to state: q1 (Starting addition of 1 for 2's complement)")

    result = list(ones_comp_result)
    carry = 1  # Start with carry 1 for adding
    current_state = 'q1'
    steps.append(f"Adding 1 to the 1's Complement result to get 2's Complement")

    # Traverse from the last bit to the first, adding the carry
    for i in range(len(result) - 1, -1, -1):
        next_state = f'q{i+2}'
        if result[i] == '1' and carry == 1:
            result[i] = '0'
            steps.append(f"Input: '1' + carry -> Transition: {current_state} -> {next_state} (Result: '0' with carry)")
        elif result[i] == '0' and carry == 1:
            result[i] = '1'
            carry = 0
            steps.append(f"Input: '0' + carry -> Transition: {current_state} -> {next_state} (Result: '1', no carry)")
        else:
            steps.append(f"No addition -> Transition: {current_state} -> {next_state} (Result: '{result[i]}')")
        current_state = next_state

    if carry == 1:
        result.insert(0, '1')
        steps.append("Final carry resulted in extra bit: '1' added at the beginning")

    twos_comp = ''.join(result)
    steps.append(f"End in final state: {current_state} (2's Complement result: {twos_comp})")
    return "\n".join(steps)

# gradio
def complement_calculator_dfa(binary_str, complement_type):
    if complement_type == "1's Complement":
        _, steps = ones_complement_dfa(binary_str)
        return steps
    elif complement_type == "2's Complement":
        return twos_complement_dfa(binary_str)
    else:
        return "Invalid Selection"

iface = gr.Interface(
    fn=complement_calculator_dfa,
    inputs=[
        gr.Textbox(label="Enter a binary string"),
        gr.Radio(["1's Complement", "2's Complement"], label="Select Complement Type")
    ],
    outputs="text",
    title="Binary Complement Calculator with Turing machine "
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f352336ba5d50ad7dc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
